# VQE: An algorithm to find the correct ground state energy of a molecule using a tranformation to a qubit simulation and the variational method.

We aim to find the approximate ground state energy and wavefunction of the $H_2$ molecule by minimising cost = $E = \langle \psi |\hat H_e | \psi\rangle$.

In [50]:
import pennylane as qml
import pennylane.numpy as np # or from pennylane import numpy as np
from pennylane import qchem
from pennylane.devices.default_qubit import DefaultQubit

symbols = ["H", "H"]
a = 1.3228 # (equilibrium?) bond length.
nuclear_coords = np.array([0.0, 0.0, -a/2, 0.0, 0.0, a/2]) # one H at 0,0,-a/2, other H at 0,0,a/2

H, qubits = qchem.molecular_hamiltonian(symbols=symbols, coordinates=nuclear_coords, mapping="jordan_wigner")
# H is the molecule's hamiltonian, qubits is number of qubits required to simulate a circuit that can give us <H>

print("Qubits required:", qubits, "\n")
print("Hamiltonian:", H) # H is written as a linear combination of Pauli operators with the indices next to the operator being the qubit they act on. Cool!

Qubits required: 4 

Hamiltonian:   (-0.24274501250441338) [Z2]
+ (-0.24274501250441338) [Z3]
+ (-0.04207255204049776) [I0]
+ (0.17771358235511994) [Z0]
+ (0.17771358235511997) [Z1]
+ (0.12293330446063176) [Z0 Z2]
+ (0.12293330446063176) [Z1 Z3]
+ (0.1676833885118713) [Z0 Z3]
+ (0.1676833885118713) [Z1 Z2]
+ (0.1705975927542705) [Z0 Z1]
+ (0.17627661386357585) [Z2 Z3]
+ (-0.04475008405123956) [Y0 Y1 X2 X3]
+ (-0.04475008405123956) [X0 X1 Y2 Y3]
+ (0.04475008405123956) [Y0 X1 X2 Y3]
+ (0.04475008405123956) [X0 Y1 Y2 X3]


In [51]:
dev = DefaultQubit(wires=qubits)
classical_opt = qml.GradientDescentOptimizer(stepsize=0.4) # stepsize = alpha?

def circuit(theta):
    qml.BasisState(np.array([1, 1, 0, 0]), wires=range(qubits))
    qml.DoubleExcitation(theta, wires=range(4)) # takes 0011-> c(theta/2)0011 + s1100, 1100 -> -s0011 + c1100

# the cost function
@qml.qnode(dev)
def cost(theta): 
    circuit(theta); return qml.expval(H)

In [53]:
# now the gradient(of cost_fn = expectation val - which is computed by dev) descent
def optimize(tol: float=1e-6, max_iter: int=100):
    theta = np.array(0, requires_grad=True) # initially psi = 1100
    energies = [cost(theta)]; thetas = [theta]

    for _ in range(max_iter):
        theta_updated = classical_opt.step(cost, thetas[-1])
        # classical_opt.step_and_cost()
        # grad, _ = classical_opt.compute_grad(cost, [thetas[:-1]], thetas[:-1])
        # theta_updated = classical_opt.apply_grad(grad, [thetas[:-1]])
        
        if abs(cost(theta_updated) - energies[-1]) < tol: break

        thetas.append(theta_updated)
        energies.append(cost(theta_updated))
    return thetas, energies

thetas, energies = optimize()
thetas = [float(theta) for theta in thetas]
energies = [float(energy) for energy in energies]
for i, (theta, energy) in enumerate(zip(thetas, energies)):
    print(f"{i:>2}th iteration: Wave function = {np.cos(theta/2):.4f}|1100> - {np.sin(theta/2):.4f}|0011>, Energy = {energy*1000:.3f} mHa")

 0th iteration: Wave function = 1.0000|1100> - 0.0000|0011>, Energy = -1117.349 mHa
 1th iteration: Wave function = 0.9994|1100> - 0.0358|0011>, Energy = -1128.000 mHa
 2th iteration: Wave function = 0.9982|1100> - 0.0594|0011>, Energy = -1132.649 mHa
 3th iteration: Wave function = 0.9972|1100> - 0.0750|0011>, Energy = -1134.662 mHa
 4th iteration: Wave function = 0.9964|1100> - 0.0852|0011>, Energy = -1135.531 mHa
 5th iteration: Wave function = 0.9958|1100> - 0.0919|0011>, Energy = -1135.906 mHa
 6th iteration: Wave function = 0.9954|1100> - 0.0963|0011>, Energy = -1136.067 mHa
 7th iteration: Wave function = 0.9951|1100> - 0.0992|0011>, Energy = -1136.137 mHa
 8th iteration: Wave function = 0.9949|1100> - 0.1011|0011>, Energy = -1136.167 mHa
 9th iteration: Wave function = 0.9948|1100> - 0.1023|0011>, Energy = -1136.179 mHa
10th iteration: Wave function = 0.9947|1100> - 0.1031|0011>, Energy = -1136.185 mHa
11th iteration: Wave function = 0.9946|1100> - 0.1037|0011>, Energy = -1136.